In [1]:
import requests
from html import unescape
import re
from bs4 import BeautifulSoup

import sys
sys.path.append('./helpers')
from preprocessing import preprocess_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pwd

In [ ]:
url = 'https://www.bbc.com/sitemaps/https-index-com-news.xml'

In [ ]:
res = requests.get(url)
if res.status_code != 200:
    print('ohho')

soup = BeautifulSoup(res.text, 'xml')

sitemap_tags = soup.find_all('sitemap')

sitemaps = []
for sitemap in sitemap_tags:
    loc = sitemap.find('loc')
    sitemaps.append(loc.text)

urls = []
regex_1 = r"^https://www.bbc.com/news/.*$"
regex_2 = r"^https://www.bbc.com/sport/.*$"
for sitemap in sitemaps:
    response = requests.get(sitemap)
    soup = BeautifulSoup(response.text, 'xml')
    url_tags = soup.find_all('url')
    
    for url in url_tags:
        loc = url.find('loc')
        if re.match(regex_1, loc.text) or re.match(regex_2, loc.text):
            urls.append(loc.text)

In [ ]:
url = urls[0]

In [ ]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        article_clean = preprocess_text(full_article)
        return (url,heading, article_clean) 
    except requests.exceptions.Timeout as ee :
        return None

In [ ]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('ohho')
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        
        full_article_part = full_article[:600]
        article_clean = preprocess_text(full_article)
        return (url,heading, article_clean,full_article_part )
    
    except requests.exceptions.Timeout as ee :
        print('TO')
        return None

In [ ]:
url = 'https://www.bbc.com/news/uk-england-humber-65173206'

In [ ]:
url_to_text(url)[3]

In [ ]:
import psycopg2
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


In [ ]:
query = f"SELECT * FROM sites where id = 1;"

In [ ]:
    cursor.execute(query)
    results = cursor.fetchall()

In [ ]:
for row in results:
    print(row)

In [ ]:
(row[5])

## disco 

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()
def reated_documents(user_input):
    words = user_input.split()
    rel_docs = set()
    for word in words:
        myquery = {"word": word}
        results  = mycollection.find(myquery)
        for result in results:
            docs = result['doc_ids']
            rel_docs.update(docs)
            
    return list(rel_docs)

def url_documents(rel_docs):
    documents  = []
    urls = []
    header = []
    raw_text = []
    for idx in rel_docs:
        query = f"SELECT * FROM sites where id = {idx};"
        cursor.execute(query)
        results = cursor.fetchall()
        for row in results:
            doc_id = row[0]
            url = row[1]
            urls.append(url)
            text = row[2]
            header = row[4]
            raw_text = row[5]
            documents.append(text)
    return (documents, urls, header,raw_text )



In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
user_query = 'london India'

In [ ]:
user_input = preprocess_text(user_query)
print(user_input)

In [ ]:
rel_docs = reated_documents(user_input)

In [ ]:
documents , urls, header, raw_text = url_documents(rel_docs)

In [ ]:
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
tokenized_query = user_query.split()

In [ ]:
header

In [ ]:
doc_scores_raw = bm25.get_scores(tokenized_query)

In [ ]:
len(doc_scores_raw)

In [ ]:
for x, y in zip(doc_scores_raw/sum(doc_scores_raw),doc_scores_raw/274):
    print(x,y,x>y)

In [2]:
sys.path.append('./models')

In [3]:
from models.bm25_base import BM25_json

In [12]:
BM25_json('bitch')

({1: {'rank': 1,
   'doc_id': 1424,
   'score': 1.0,
   'url': 'https://www.bbc.com/news/technology-64989720',
   'header': "How Elon Musk's tweets unleashed a wave of hate",
   'text': 'How Elon Musk\'s tweets unleashed a wave of hate I had just finished my investigation into whether Twitter can protect users under Elon Musk\'s ownership, when - to my surprise - the man himself tweeted about it. "Sorry for turning Twitter from nurturing paradise into a place that hasâ\x80¦ trolls," he said in one tweet, posting a screengrab of the report.  According to Twitter\'s own data, that tweet was seen by more than 30 million profiles. "Trolls are kinda fun," Mr Musk said in another reply, in his response to my BBC investigation - Twitter insiders: We can\'t protect users from trolling under ',
   'session_id': 1}},
 [1.0])

In [8]:
from models.desm import DESM_json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [11]:
DESM_json('bitch')

C:\Users\iq\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:620: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


({1: {'rank': 1,
   'doc_id': 1424,
   'score': 1.0,
   'url': 'https://www.bbc.com/news/technology-64989720',
   'header': "How Elon Musk's tweets unleashed a wave of hate",
   'text': 'How Elon Musk\'s tweets unleashed a wave of hate I had just finished my investigation into whether Twitter can protect users under Elon Musk\'s ownership, when - to my surprise - the man himself tweeted about it. "Sorry for turning Twitter from nurturing paradise into a place that hasâ\x80¦ trolls," he said in one tweet, posting a screengrab of the report.  According to Twitter\'s own data, that tweet was seen by more than 30 million profiles. "Trolls are kinda fun," Mr Musk said in another reply, in his response to my BBC investigation - Twitter insiders: We can\'t protect users from trolling under ',
   'session_id': 1}},
 [1.0])